In [147]:
import pypandoc
import requests
from bs4 import BeautifulSoup
import unicodedata
import re

In [148]:
output=pypandoc.convert_file('testdocs/Coordinates of a Point in Space.docx','html')
soup=BeautifulSoup(output,'html.parser')

In [149]:
print(soup.prettify())

<table>
 <colgroup>
  <col style="width: 49%"/>
  <col style="width: 50%"/>
 </colgroup>
 <tbody>
  <tr class="odd">
   <td>
    Division
   </td>
   <td>
    11
   </td>
  </tr>
  <tr class="even">
   <td>
    Subject
   </td>
   <td>
    Maths
   </td>
  </tr>
  <tr class="odd">
   <td>
    Category
   </td>
   <td>
    Very Easy
   </td>
  </tr>
  <tr class="even">
   <td>
    No of mcq
   </td>
   <td>
    15
   </td>
  </tr>
  <tr class="odd">
   <td>
    Author
   </td>
   <td>
    Auto Scrapper
   </td>
  </tr>
 </tbody>
</table>
<table>
 <colgroup>
  <col style="width: 100%"/>
 </colgroup>
 <tbody>
  <tr class="odd">
   <td>
    The point (4, 0, 0) lie on _____________
   </td>
  </tr>
  <tr class="even">
   <td>
    X-axis
   </td>
  </tr>
  <tr class="odd">
   <td>
    Y-axis
   </td>
  </tr>
  <tr class="even">
   <td>
    Z-axis
   </td>
  </tr>
  <tr class="odd">
   <td>
    Y-Z plane
   </td>
  </tr>
  <tr class="even">
   <td>
    1
   </td>
  </tr>
  <tr class="odd">
  

In [150]:
config=soup.find_all('table')[0]
questions=soup.find_all('table')[1:]
no_questions=len(questions)
no_questions

15

In [151]:
# config=pd.read_html(str(config))[0]
# config
questions[0]

<table>
<colgroup>
<col style="width: 100%"/>
</colgroup>
<tbody>
<tr class="odd">
<td>The point (4, 0, 0) lie on _____________</td>
</tr>
<tr class="even">
<td>X-axis</td>
</tr>
<tr class="odd">
<td>Y-axis</td>
</tr>
<tr class="even">
<td>Z-axis</td>
</tr>
<tr class="odd">
<td>Y-Z plane</td>
</tr>
<tr class="even">
<td>1</td>
</tr>
<tr class="odd">
<td></td>
</tr>
<tr class="even">
<td>In 3d coordinate system, a point with y and z coordinate zero and
x-coordinate having non-zero value must lie on x-axis. So, (4, 0, 0) lie
on x-axis.</td>
</tr>
<tr class="odd">
<td>Three Dimensional Geometry</td>
</tr>
</tbody>
</table>

In [152]:
question_set={}
format=["Question","Option A","Option B","Option C","Option D","Correct Answer","Hint","Explanation","Sub-topic"]

In [153]:
rows = config.find_all('tr')
table_dict = {cols[0].text: cols[1].text for row in rows for cols in [row.find_all(['th', 'td'])] if cols}
# print(table_dict)
question_set["Config"]=table_dict
question_set

{'Config': {'Division': '11',
  'Subject': 'Maths',
  'Category': 'Very Easy',
  'No of mcq': '15',
  'Author': 'Auto Scrapper'}}

In [154]:
print(questions[1])

<table>
<colgroup>
<col style="width: 100%"/>
</colgroup>
<tbody>
<tr class="odd">
<td>The point (0, 0, 3) lie on ____________</td>
</tr>
<tr class="even">
<td>X-axis</td>
</tr>
<tr class="odd">
<td>Y-axis</td>
</tr>
<tr class="even">
<td>Z-axis</td>
</tr>
<tr class="odd">
<td>X-Y plane</td>
</tr>
<tr class="even">
<td>3</td>
</tr>
<tr class="odd">
<td></td>
</tr>
<tr class="even">
<td>In 3d coordinate system, a point with x and y coordinate zero and
z-coordinate having non-zero value must lie on z-axis. So, (0, 0, 3) lie
on z-axis.</td>
</tr>
<tr class="odd">
<td>Three Dimensional Geometry</td>
</tr>
</tbody>
</table>


In [177]:
def extract_data(question):
    question_dict = {}
    format = ["Question", "Option A", "Option B", "Option C", "Option D", "Correct Answer", "Hint", "Explanation", "Sub-topic"]

    # Extract the question
    # p_tags = question.find('th').find_all('p')
    # question_text = ' '.join([p.text for p in p_tags[:-1]])  # Join all p tags except the last one (year)
    try:
        question_text=question.find('tr',class_='header').text.strip()
        question_text = re.sub(r'\(\d+\)', '', question_text)  # Remove the year

        question_dict[format[0]] = question_text.strip().replace('\n', ' ')

        # Extract the options
        options = question.find_all('td')
        for i, option in enumerate(options[:5], 1):  # Only the first 5 td tags are options
            if option.find('blockquote'):  # If the option is in a blockquote tag
                text = option.find('blockquote').text.strip().replace('\n', ' ')
            elif option.find('p'):  # If the option is in a p tag
                text = option.find('p').text.strip().replace('\n', ' ')
            else:  # If the option is directly in the td tag
                text = option.text.strip().replace('\n', ' ')
            question_dict[format[i]] = text

        # Extract the correct answer
        mapping = {'1': 'a', 'a': 'a', '2': 'b', 'b': 'b', '3': 'c', 'c': 'c', '4': 'd', 'd': 'd'}

        correct_answer = options[4].text.strip().replace('\n', ' ')
        question_dict[format[5]] = mapping.get(correct_answer,None)

        # Extract the hint (assuming it's the text in the 7th td tag)
        hint = options[5].text.strip().replace('\n', ' ')
        question_dict[format[6]] = hint

        # Extract the explanation
        explanation = options[6]  # Explanation is in multiple p tags
        explanation_text = ' '.join([p.text for p in explanation]).strip().replace('\n', ' ')
        question_dict[format[7]] = explanation_text

        # Extract the sub-topic
        sub_topic = options[7].text.strip().replace('\n', ' ')
        question_dict[format[8]] = sub_topic

        return question_dict
    except:
        for i , content in enumerate(question.find_all('tr')):
            question_dict[format[i]]=content.text.strip()
        return question_dict


    
extract_data(questions[0])


{'Question': 'The point (4, 0, 0) lie on _____________',
 'Option A': 'X-axis',
 'Option B': 'Y-axis',
 'Option C': 'Z-axis',
 'Option D': 'Y-Z plane',
 'Correct Answer': '1',
 'Hint': '',
 'Explanation': 'In 3d coordinate system, a point with y and z coordinate zero and\nx-coordinate having non-zero value must lie on x-axis. So, (4, 0, 0) lie\non x-axis.',
 'Sub-topic': 'Three Dimensional Geometry'}

In [160]:
cont='''
<table>
<colgroup>
<col style="width: 100%"/>
</colgroup>
<tbody>
<tr class="odd">
<td>The point (0, 0, 3) lie on ____________</td>
</tr>
<tr class="even">
<td>X-axis</td>
</tr>
<tr class="odd">
<td>Y-axis</td>
</tr>
<tr class="even">
<td>Z-axis</td>
</tr>
<tr class="odd">
<td>X-Y plane</td>
</tr>
<tr class="even">
<td>3</td>
</tr>
<tr class="odd">
<td></td>
</tr>
<tr class="even">
<td>In 3d coordinate system, a point with x and y coordinate zero and
z-coordinate having non-zero value must lie on z-axis. So, (0, 0, 3) lie
on z-axis.</td>
</tr>
<tr class="odd">
<td>Three Dimensional Geometry</td>
</tr>
</tbody>
</table>
'''
soup=BeautifulSoup(cont,'html.parser')
# extract_data(BeautifulSoup(cont))

In [172]:
for i in soup.find_all('tr'):
    print(i.text.strip())

The point (0, 0, 3) lie on ____________
X-axis
Y-axis
Z-axis
X-Y plane
3

In 3d coordinate system, a point with x and y coordinate zero and
z-coordinate having non-zero value must lie on z-axis. So, (0, 0, 3) lie
on z-axis.
Three Dimensional Geometry


In [178]:
question_set["Questions"]=[extract_data(question) for question in questions]

In [179]:
extract_data(questions[1])

# counter=0
# for question in questions:
#     counter+=1
#     print(counter)
#     print(extract_data(question))

{'Question': 'The point (0, 0, 3) lie on ____________',
 'Option A': 'X-axis',
 'Option B': 'Y-axis',
 'Option C': 'Z-axis',
 'Option D': 'X-Y plane',
 'Correct Answer': '3',
 'Hint': '',
 'Explanation': 'In 3d coordinate system, a point with x and y coordinate zero and\nz-coordinate having non-zero value must lie on z-axis. So, (0, 0, 3) lie\non z-axis.',
 'Sub-topic': 'Three Dimensional Geometry'}

In [129]:
sel_question=questions[47]
# def extqs(sel_question):
# Extract the question
# ques=sel_question.find('thead').find('p').text.strip().replace('\n', ' ') 
# for i in questions:
p_tags = sel_question.find('th').find_all('p')
ques = ' '.join([p.text for p in p_tags[:-1]])  # Join all p tags except the last one (year) 
ques = re.sub(r'\(\d+\)', '', ques)  # Remove the year 
# print(f"Question {questions.index(i)} : {ques}") 

opta=sel_question.find('tbody').find_all('tr')[0].text.strip().replace('\n', ' ')
optb=sel_question.find('tbody').find_all('tr')[1].text.strip().replace('\n', ' ')
optc=sel_question.find('tbody').find_all('tr')[2].text.strip().replace('\n', ' ')
optd=sel_question.find('tbody').find_all('tr')[3].text.strip().replace('\n', ' ')
coropt=sel_question.find('tbody').find_all('tr')[4].text.strip().replace('\n', ' ')
hint=sel_question.find('tbody').find_all('tr')[5].text.strip().replace('\n', ' ') 
exp=sel_question.find('tbody').find_all('tr')[6].text.strip().replace('\n', ' ') 
subt=sel_question.find('tbody').find_all('tr')[7].text.strip().replace('\n', ' ')
data=[ques,opta,optb,optc,optd,coropt,hint,exp,subt]

for i in questions:
    print(f"""Question {questions.index(i)} : {i.find('tr',class_='header').text.strip() }  """ )

Question 0 : Which among the following represent intercepted in Arrhenius
plot
(2012)  
Question 1 : Half-life period in first order reaction depends upon
(2011)  
Question 2 : What would be the unit of k in the given reaction
K
A +B → C and the given reaction is a homogeneous reaction
(2015)  
Question 3 : Which among the following can increase the rate constant of
reaction
(2010)  
Question 4 : The given reaction is
2N₂O₅ ⇔2N₂O₄ + O₂
(2008)  
Question 5 : Calculate the half-life period of a first order reaction where
specific rate constant is 2min⁻¹
(2003)  
Question 6 : In Arrhenius plot slope Is
(2002)  
Question 7 : Find the time taken for completing half reaction if the initial
concentration of zero order reaction is reduced to 1/4thh
(2004)  
Question 8 : For the zero-order reaction rate constant is equals to
(2017)  
Question 9 : Half-life for second order reaction is
(2003)  
Question 10 : Rate constant for first order reaction is 2.6 × 10⁻¹⁴ s⁻¹. Find
the half-life period of 

In [21]:
question_set=dict(zip(format,extqs(sel_question)))

In [26]:
# question_set
extqs(sel_question)

['The given reaction is', 'Unimolecular', 'Multimolecular', 'Bimolecular', 'None of the above', 'c', 'The number of reacting species taking part in an elementary reaction which must collide simultaneously in order to bring about chemical reaction is called molecularity of a reaction.', 'The given reaction 2N₂O₅ ⇔2N₂O₄ + O₂ It is bimolecular and first order reaction because rate is directly proportional to the reactant [N₂O₅].', 'Instantaneous rate']


In [23]:
questions[4].find('tbody')

<tbody>
<tr class="odd">
<td><blockquote>
<p>Unimolecular</p>
</blockquote></td>
</tr>
<tr class="even">
<td><blockquote>
<p>Multimolecular</p>
</blockquote></td>
</tr>
<tr class="odd">
<td><blockquote>
<p>Bimolecular</p>
</blockquote></td>
</tr>
<tr class="even">
<td><blockquote>
<p>None of the above</p>
</blockquote></td>
</tr>
<tr class="odd">
<td>c</td>
</tr>
<tr class="even">
<td>The number of reacting species taking part in an elementary reaction
which must collide simultaneously in order to bring about chemical
reaction is called molecularity of a reaction.</td>
</tr>
<tr class="odd">
<td><p>The given reaction 2N₂O₅ ⇔2N₂O₄ + O₂</p>
<p>It is bimolecular and first order reaction because rate is directly
proportional to the reactant [N₂O₅].</p></td>
</tr>
<tr class="even">
<td>Instantaneous rate</td>
</tr>
</tbody>